In [ ]:
from IPython.display import display
import ipywidgets as ipw
import widgets
import utils
import os

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
DATA_MODEL = utils.read_yaml("/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])
SIMULATION_OBJECT_CLASSES = ["BandStructure", "GeometryOptimisation", "PDOS", "MeasurementSession", "UnclassifiedSimulation", "VibrationalSpectroscopy"]
SIMULATION_OBJECT_TYPES = []
for simulation in SIMULATION_OBJECT_CLASSES:
    object_type = DATA_MODEL["classes"][simulation]["annotations"]["openbis_label"].replace(" ", "_").upper()
    SIMULATION_OBJECT_TYPES.append(object_type)

project_selector = widgets.ProjectSelectionWidget()
project_selector.load_dropdown_box()

measurements_selector = widgets.ObjectMultipleSelectionWidget("Measurements")
simulations_selector = widgets.ObjectMultipleSelectionWidget("Simulations")
analysis_selector = widgets.ObjectMultipleSelectionWidget("Analysis")
results_widgets = widgets.OpenbisObjectWidget("Results", DATA_MODEL)

results_support_files = ipw.FileUpload(multiple = True)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def load_project_objects_metadata(change):
    if project_selector.dropdown.value != -1:
        simulations_selector.load_selector(SIMULATION_OBJECT_TYPES, project_selector.dropdown.value)
        analysis_selector.load_selector("ANALYSIS", project_selector.dropdown.value)
        measurements_selector.load_selector(["MEASUREMENT_SESSION"], project_selector.dropdown.value)

def create_results_openbis(b):
    project_permid = project_selector.dropdown.value
    project_results_collection = OPENBIS_SESSION.get_collections(project = project_permid, code = "RESULTS_COLLECTION")
    results_parents = []
    results_parents.extend(list(measurements_selector.selector.value))
    results_parents.extend(list(analysis_selector.selector.value))
    results_parents.extend(list(simulations_selector.selector.value))
    
    if len(project_results_collection) == 0:
        results_collection = utils.create_openbis_collection(
            OPENBIS_SESSION, 
            project = project_selector.dropdown.value, 
            code = "RESULTS_COLLECTION", type = "COLLECTION", props = {"name": "Results"}
        )
    else:
        results_collection = project_results_collection[0]

    object_properties = utils.get_object_widget_values(results_widgets.properties_widgets)
    results_object = utils.create_openbis_object(
        OPENBIS_SESSION, type="RESULTS", collection=results_collection, 
        props = object_properties, parents = results_parents
    )
    utils.upload_datasets(OPENBIS_SESSION, results_object, results_support_files, "RAW_DATA")
    display(utils.Javascript(data = "alert('Upload successful!')"))
    
    # Reset widgets
    measurements_selector.selector.options = []
    simulations_selector.selector.options = []
    analysis_selector.selector.options = []
    project_selector.dropdown.value = -1
    results_widgets.reset_properties_widgets()

# Create results

## Select project

In [ ]:
display(project_selector.dropdown_boxes)
project_selector.dropdown.observe(load_project_objects_metadata, names = 'value')

## Select analysis

In [ ]:
display(analysis_selector)

## Select simulations

In [ ]:
display(simulations_selector)

## Select measurements

In [ ]:
display(measurements_selector)

## Properties

In [ ]:
display(results_widgets)

## Support files

In [ ]:
display(results_support_files)

## Save results

In [ ]:
display(save_close_buttons_hbox)
display(increase_buttons_size)
create_button.on_click(create_results_openbis)
quit_button.on_click(close_notebook)